### 参考notebook
https://www.kaggle.com/code/debarshichanda/pytorch-w-b-jigsaw-starter

In [ ]:
# デバッグ用コード追加
#debug = True
debug = False

In [ ]:
!nvidia-smi

Fri Apr 29 00:52:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --upgrade wandb

In [ ]:
# albertのtokenizationに必要
!pip install --upgrade pip
!pip install transformers[sentencepiece]

In [ ]:
# albertのtokenizationに必要
import sentencepiece

In [ ]:
!pip install GPUtil
!pip install transformers==4.12.2
!pip install fugashi
!pip install mecab-python3
!pip install ipadic

!pip install colorama

In [ ]:
import os
import gc
import copy
import time
import random
import string

import datetime
from datetime import datetime, timedelta, timezone

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import re
import unicodedata

import GPUtil
import regex
import scipy as sp
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, AdamW
  
warnings.filterwarnings('ignore')

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: kuroyuli (use `wandb login --relogin` to force relogin)


True

In [ ]:
def id_generator(size=12, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
current_time = now.strftime("%Y-%m-%d-%H-%M", )

HASH_NAME = "Fakenews-"+current_time

print(HASH_NAME)

Fakenews-2022-04-29-09-53


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ディレクトリ名を変更しました
ROOT = "/content/drive/MyDrive/Colab Notebooks/fakenews"
CUR_DIR = ROOT+"/baseline/code"
%cd $CUR_DIR

/content/drive/MyDrive/Colab Notebooks/fakenews/baseline/code


In [ ]:
CONFIG = {"seed": 2022,
          "epochs": 10, # OneCycleLR用に少なめ
          "model_name": r'ALINEAR/albert-japanese-v2',  # r'cl-tohoku/bert-base-japanese-v2'から変更
          "train_batch_size": 64,  # albertは軽いのでバッチサイズを上げても動く。ただし、colabでメモリが少ないマシンを
          "valid_batch_size": 64,  # 割り当てられてしまった場合は、時間をあけて再トライするか、バッチを64に下げる。
          "max_length": 128,
          "learning_rate": 1e-5,    # OneCycleLR用に比較的大きい学習率を設定
          "scheduler": 'OneCycleLR',  # CosineAnnealingLRから変更
          "total_steps":230,  # OneCycleLR用（trainのサイズ/バッチサイズ×エポック数）
          "min_lr": 3e-7,
          "T_max": 5,
          "weight_decay": 1e-6,
          "max_grad_norm" :1.0,
          "n_accumulate": 1,
          "num_classes": 2,
          "n_fold": 5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": HASH_NAME,
          "hidden_size" :768,
          "num_hidden_layers": 24,    
          "dropout" :0.2
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
CONFIG['group'] = f'{HASH_NAME}-Baseline'

if debug == True:
  CONFIG["epochs"] = 1

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
df = pd.read_csv(ROOT+"/data/train.csv")

In [ ]:
if debug == True:
  df = df[:512]

In [ ]:
#df["isFake"].value_counts()

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Folds</h1></span>

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df["isFake"])):
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
#df.head()

In [ ]:
class FakenewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        self.target = df['isFake'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
    
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
    
        target = self.target[index]
        
        onehot_t = np.zeros(CONFIG["num_classes"], dtype=np.float32) + 0.0025 ### Label smoothing
        onehot_t[target] = 0.995
        
        ids = inputs_text['input_ids']
        mask = inputs_text['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(onehot_t, dtype=torch.float)
        }

In [ ]:
class FakenewsModel(nn.Module):
    def __init__(self, model_name):
        super(FakenewsModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        outputs = self.sigmoid(outputs)
        
        return outputs.squeeze()

In [ ]:
def criterion(outputs, targets):
    loss_f = nn.BCELoss()
    """
    print("output",outputs.shape, 'targets',targets.shape)
    print(outputs)
    print(targets)
    """
    return loss_f(outputs,targets)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold, savepath):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            #PATH = f"Loss-Fold-{fold}.bin"    ####ここのPATHは変えよう
            PATH = savepath + f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = FakenewsDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = FakenewsDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    # OneCycleLRを追加
    elif CONFIG['scheduler'] == 'OneCycleLR':
        scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=CONFIG['learning_rate'],
                                            total_steps=CONFIG['total_steps'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
# パス名変更
modelpath = f"{ROOT}/baseline/models/albert-{current_time}/"  
if not os.path.isdir(modelpath):
    os.makedirs(modelpath)

In [ ]:
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    run = wandb.init(project='Fakenews', 
                     config=CONFIG,
                     job_type='Train',
                     group=CONFIG['group'],
                     tags=['albert', f'{HASH_NAME}', 'bceloss'],
                     name=f'{HASH_NAME}-fold-{fold}',
                     anonymous='must')
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = FakenewsModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold,
                                  savepath=modelpath)
    
    run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()

====== Fold: 0 ======


Some weights of the model checkpoint at ALINEAR/albert-japanese-v2 were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: Tesla T4



 38%|███▊      | 18/47 [00:23<00:37,  1.29s/it, Epoch=1, LR=1.97e-6, Train_Loss=0.705]


KeyboardInterrupt: ignored

## Validation

In [ ]:
!ls "$ROOT/baseline/models/"

In [ ]:
from glob import glob
MODEL_PATHS = glob(f'{modelpath}/*.bin')
MODEL_PATHS.sort()

In [ ]:
MODEL_PATHS

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        
        #PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
        PREDS.append(outputs.cpu().detach().numpy()) 

    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FakenewsModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
def get_labels_frompred(out, multi=False):

    print(out)
    o = (-out).argsort(1) 
    preds = int(o[0][0].cpu().numpy())
    print(preds)

    return preds

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

fold_f1scores = []
fold_accuracyscores = []

for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    
    train_loader, valid_loader = prepare_loaders(fold=fold)

    valid = df[df["kfold"]==fold]

    out = inference([MODEL_PATHS[fold]], valid_loader, CONFIG['device'])

    valid["preds"] = np.argmax(out, axis=1)

    fold_f1scores.append(f1_score(valid["isFake"].values, valid["preds"].values))
    fold_accuracyscores.append(accuracy_score(valid["isFake"].values, valid["preds"].values))


In [ ]:
fold_f1scores

In [ ]:
fold_accuracyscores